## This notebook contains a sample code for the COMPAS data experiment in Section 5.2.

Before running the code, please check README.md and install LEMON.

In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn import feature_extraction
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import stealth_sampling

### Functions

In [2]:
# split data to bins (s, y) = (1, 1), (1, 0), (0, 1), (0, 0)
def split_to_four(X, S, Y):
    Z = np.c_[X, S, Y]
    Z_pos_pos = Z[np.logical_and(S, Y), :]
    Z_pos_neg = Z[np.logical_and(S, np.logical_not(Y)), :]
    Z_neg_pos = Z[np.logical_and(np.logical_not(S), Y), :]
    Z_neg_neg = Z[np.logical_and(np.logical_not(S), np.logical_not(Y)), :]
    Z = [Z_pos_pos, Z_pos_neg, Z_neg_pos, Z_neg_neg]
    return Z

# compute demographic parity
def demographic_parity(W):
    p_pos = np.mean(np.concatenate(W[:2]))
    p_neg = np.mean(np.concatenate(W[2:]))
    return np.abs(p_pos - p_neg)

# compute the sampling size from each bin
def computeK(Z, Nsample, sampled_spos, sampled_ypos):
    Kpp = Nsample*sampled_spos*sampled_ypos[0]
    Kpn = Nsample*sampled_spos*(1-sampled_ypos[0])
    Knp = Nsample*(1-sampled_spos)*sampled_ypos[1]
    Knn = Nsample*(1-sampled_spos)*(1-sampled_ypos[1])
    K = [Kpp, Kpn, Knp, Knn]
    kratio = min([min(1, z.shape[0]/k) for (z, k) in zip(Z, K)])
    Kpp = int(np.floor(Nsample*kratio*sampled_spos*sampled_ypos[0]))
    Kpn = int(np.floor(Nsample*kratio*sampled_spos*(1-sampled_ypos[0])))
    Knp = int(np.floor(Nsample*kratio*(1-sampled_spos)*sampled_ypos[1]))
    Knn = int(np.floor(Nsample*kratio*(1-sampled_spos)*(1-sampled_ypos[1])))
    K = [max([k, 1]) for k in [Kpp, Kpn, Knp, Knn]]
    return K

# case-contrl sampling
def case_control_sampling(X, K):
    q = [(K[i]/sum(K)) * np.ones(x.shape[0]) / x.shape[0] for i, x in enumerate(X)]
    return q

# compute wasserstein distance
def compute_wasserstein(X1, S1, X2, S2, timeout=10.0):
    dx = stealth_sampling.compute_wasserstein(X1, X2, path='./', prefix='compas', timeout=timeout)
    dx_s1 = stealth_sampling.compute_wasserstein(X1[S1>0.5, :], X2[S2>0.5, :], path='./', prefix='compas', timeout=timeout)
    dx_s0 = stealth_sampling.compute_wasserstein(X1[S1<0.5, :], X2[S2<0.5, :], path='./', prefix='compas', timeout=timeout)
    return dx, dx_s1, dx_s0

### Fetch data and preprocess
We modified [https://github.com/mbilalzafar/fair-classification/blob/master/disparate_mistreatment/propublica_compas_data_demo/load_compas_data.py]

In [3]:
url = 'https://raw.githubusercontent.com/propublica/compas-analysis/master/compas-scores-two-years.csv'
feature_list = ['age_cat', 'race', 'sex', 'priors_count', 'c_charge_degree', 'two_year_recid']
sensitive = 'race'
label = 'score_text'

# fetch data
df = pd.read_table(url, sep=',')
df = df.dropna(subset=['days_b_screening_arrest'])

# convert to np array
data = df.to_dict('list')
for k in data.keys():
    data[k] = np.array(data[k])

# filtering records
idx = np.logical_and(data['days_b_screening_arrest']<=30, data['days_b_screening_arrest']>=-30)
idx = np.logical_and(idx, data['is_recid'] != -1)
idx = np.logical_and(idx, data['c_charge_degree'] != 'O')
idx = np.logical_and(idx, data['score_text'] != 'NA')
idx = np.logical_and(idx, np.logical_or(data['race'] == 'African-American', data['race'] == 'Caucasian'))
for k in data.keys():
    data[k] = data[k][idx]
    
# label Y
Y = 1 - np.logical_not(data[label]=='Low').astype(np.int32)

# feature X, sensitive feature S
X = []
for feature in feature_list:
    vals = data[feature]
    if feature == 'priors_count':
        vals = [float(v) for v in vals]
        vals = preprocessing.scale(vals)
        vals = np.reshape(vals, (Y.size, -1))
    else:
        lb = preprocessing.LabelBinarizer()
        lb.fit(vals)
        vals = lb.transform(vals)
    if feature == sensitive:
        S = vals[:, 0]
    X.append(vals)
X = np.concatenate(X, axis=1)

### Experiment

In [4]:
# parameter settings
seed = 0                    # random seed

# parameter settings for sampling
Nsample = 2000              # number of data to sample
sampled_ypos = [0.5, 0.5]   # the ratio of positive decisions '\alpha' in sampling

# parameter settings for complainer
Nref = 1278                 # number of referential data

In [5]:
def sample_and_evaluate(X, S, Y, Nref=1278, Nsample=2000, sampled_ypos=[0.5, 0.5], seed=0):
    
    # load data
    Xbase, Xref, Sbase, Sref, Ybase, Yref = train_test_split(X, S, Y, test_size=Nref, random_state=seed)
    N = Xbase.shape[0]
    scaler = StandardScaler()
    scaler.fit(Xbase)
    Xbase = scaler.transform(Xbase)
    Xref = scaler.transform(Xref)

    # wasserstein distance between base and ref
    np.random.seed(seed)
    idx = np.random.permutation(Xbase.shape[0])[:Nsample]
    dx, dx_s1, dx_s0 = compute_wasserstein(Xbase[idx, :], Sbase[idx], Xref, Sref, timeout=10.0)

    # demographic parity
    Z = split_to_four(Xbase, Sbase, Ybase)
    parity = demographic_parity([z[:, -1] for z in Z])
    
    # sampling
    results = [[parity, dx, dx_s1, dx_s0]]
    sampled_spos = np.mean(Sbase)
    K = computeK(Z, Nsample, sampled_spos, sampled_ypos)
    for i, sampling in enumerate(['case-control', 'stealth']):
        #print('%s: sampling ...' % (sampling,), end='')
        np.random.seed(seed+i)
        if sampling == 'case-control':
            p = case_control_sampling([z[:, :-1] for z in Z], K)
        elif sampling == 'stealth':
            p = stealth_sampling.stealth_sampling([z[:, :-1] for z in Z], K, path='./', prefix='compas', timeout=30.0)
        idx = np.random.choice(N, sum(K), p=np.concatenate(p), replace=False)
        Xs = np.concatenate([z[:, :-2] for z in Z], axis=0)[idx, :]
        Ss = np.concatenate([z[:, -2] for z in Z], axis=0)[idx]
        Ts = np.concatenate([z[:, -1] for z in Z], axis=0)[idx]
        #print('done.')
        
        # demographic parity of the sampled data
        #print('%s: evaluating ...' % (sampling,), end='')
        Zs = split_to_four(Xs, Ss, Ts)
        parity = demographic_parity([z[:, -1] for z in Zs])
        
        # wasserstein disttance
        dx, dx_s1, dx_s0 = compute_wasserstein(Xs, Ss, Xref, Sref, timeout=10.0)
        #print('done.')
        
        results.append([parity, dx, dx_s1, dx_s0])
    return results

#### Experiment (One Run)

In [6]:
result = sample_and_evaluate(X, S, Y, Nref=Nref, Nsample=Nsample, sampled_ypos=sampled_ypos, seed=seed)
df = pd.DataFrame(result)
df.index = ['Baseline', 'Case-control', 'Stealth']
df.columns = ['DP', 'WD on Pr[x]', 'WD on Pr[x|s=1]', 'WD on Pr[x|s=0]']
print('Result (alpha = %.2f, seed=%d)' % (sampled_ypos[0], seed))
df

Result (alpha = 0.50, seed=0)


,PD,WD on Pr[x],WD on Pr[x|s=1],WD on Pr[x|s=0]
Baseline,0.245317,0.771823,0.69127,0.960298
Case-control,0.080242,0.960744,1.50996,0.839024
Stealth,0.002429,0.752672,1.07511,0.6794


#### Experiment (10 Runs)

In [7]:
num_itr = 10
result_all = []
for i in range(num_itr):
    result_i = sample_and_evaluate(X, S, Y, Nref=Nref, Nsample=Nsample, sampled_ypos=sampled_ypos, seed=i)
    result_all.append(result_i)
result_all = np.array(result_all)
df = pd.DataFrame(np.mean(result_all, axis=0))
df.index = ['Baseline', 'Case-control', 'Stealth']
df.columns = ['DP', 'WD on Pr[x]', 'WD on Pr[x|s=1]', 'WD on Pr[x|s=0]']
print('Average Result of %d runs (alpha = %.2f)' % (num_itr, sampled_ypos[0]))
df

Average Result of 10 runs (alpha = 0.50)


,PD,WD on Pr[x],WD on Pr[x|s=1],WD on Pr[x|s=0]
Baseline,0.244972,0.731744,0.828592,0.728760
Case-control,0.076987,0.762208,0.976887,0.799211
Stealth,0.014039,0.742240,0.873354,0.713801
